<div style="border:1px solid black; padding:10px 10px;">
    <strong>CIVIL-321 "Modélisation Numérique des Solides et Structures"</strong><br/><br/>
    <span style="text-decoration:underline;font-weight:bold;">Comment utiliser ce Jupyter Notebook?
    </span><br/><br/>
    Ce <strong>Notebook</strong> est constitué de cellules de texte et de cellule de code. Les cellules de codes doivent être  <strong>executées</strong> pour voir le résultat du programme. Certaines cellules doivent être remplies par vos soins. Pour exécuter une cellule, cliquez dessus simplement et ensuite cliquez sur le bouton "play" (<span style="font: bold 12px/30px Arial, serif;">&#9658;</span>) dans la barre de menu au dessus du notebook. Vous pouvez aussi taper la combinaison de touches <code>shift + enter</code>. Il est important d'éxécuter les cellules de code en respectant leur ordre d'arrivée dans le notebook.
</div>

On vous encourage à poser vos questions et donner votre feedback sur ce notebook sur la plateforme ED Discussion du cours accessible en cliquant sur ce bouton:
 
 
 
<div class="container" >
        <a href="https://edstem.org/eu/courses/409/discussion?category=Exercices">
            <button class="btn btn-primary btn-lg">Ed Discussion</button>
        </a>
</div>

In [ ]:
import numpy as np
import scipy.sparse.linalg
import matplotlib.pyplot as plt
from plot import *

# T3 iso-paramétrique complet

Pour cet exercice, nous allons nous construire un code iso-paramétrique complet.

## Maillage

![](Images/poutre_console.svg)
<center>*Poutre à mailler. Épaisseur $t = 10$ mm. $E = 210$ GPa, $\nu = 0.3$.*</center>

#### Question 

A partir de cette description géométrique, veuillez créer un maillage de T3, à l'aide de la fonction `meshGeo`, puis affiche le avec la fonction `plotMesh`

In [ ]:
help(meshGeo)
help(plotMesh)

In [ ]:
# Place your answer here

## Matrice de rigidité

#### Question

**La pluspart des routines ci-dessous vous sont données. Veuillez les lire avec attention!**

Pour faire fonctionner le code avec les éléments T3, il faut:

- implémenter l'intégration de Gauss de la matrice locale de rigidité (fonction `calculerMatriceRigiditeLocale`):

$$[K] = \iint_e [B]^T[D][B]det(J) ds dt$$

  ce qui comprend `calculerD` (loi de constitution), `calculerC` (calcul des gradients dans l'espace naturel), `calculerBetJ` (calcul de la matrice $B$ et de la jacobienne $J$) et enfin `calculerMatriceRigiditeLocale` qui procède à l'intégration numérique.

In [ ]:
def calculerD(contraintes_planes=True):
    
    E = 210e3 # 210GPa = 210e9 N/m^2 = 210e3 N/mm^2
    nu = 0.3    

    # On fait des contraintes planes par défaut
    if contraintes_planes:
        D = (E/(1-nu**2))* np.array(
            [[1,  nu, 0],
             [nu, 1,  0],
             [0,  0,  (1-nu)/2]])
    else:
        raise RuntimeError('Déformations planes à implémenter!')
    return D

def calculerC():
    N1s, N1t = [-1, -1]
    N2s, N2t = [1, 0]
    N3s, N3t = [0, 1]
    C = np.array([
        [N1s, 0, N2s, 0, N3s, 0],
        [N1t, 0, N2t, 0, N3t, 0],
        [0, N1s, 0, N2s, 0, N3s],
        [0, N1t, 0, N2t, 0, N3t]])
    return C

def calculerBetJ(x, noeuds):
    C = calculerC()

    grads = C@noeuds.ravel()
    J = np.array([[grads[0], grads[2]],
                  [grads[1], grads[3]]])
    
    
    # Définition de A
    A = np.array([
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 1, 1, 0]
    ])

    # Définition de la matrice contenant les Jacobiennes
    Jblock = np.zeros((4, 4))
    Jblock[:2, :2] = J
    Jblock[2:, 2:] = J
    
    Mat_J = np.linalg.inv(Jblock)
    B = A@Mat_J@C
    return B, J


def calculerMatriceRigiditeLocale(connectivite_element, coordonnees):
    
    quads = [[1/3, 1/3]]
    weights = [1/2]
    
    noeuds = coordonnees[connectivite_element.ravel(), :] # les noeuds de l'élément traité
    K_local = np.zeros((6, 6)) # 6x6 for T3 only
    
    for x_q, w_q in zip(quads, weights):
        B, J = calculerBetJ(x_q, noeuds)  # calcule B et J sur le point de quadrature x_q
        detJ = np.linalg.det(J)           # calcule det(J) pour l'intégration
        D = calculerD()                   # calcule la loi de constitution D
           
        # somme la quadrature avec le poid w_q
        K_local = K_local + w_q * B.T@D@B * detJ
        
    # l'épaisseur
    t = 10. # mm
    
    return t*K_local

- Procéder à l'assemblage de la matrice de rigidité complète qui comprend le calcul des numéros d'équation (fonction `calculerNumerosEquations`), et l'assemblage lui même (fonction `assemblerMatriceRigidite`)

In [ ]:
        
def calculerNumerosEquations(connectivite):

    n_elem  = connectivite.shape[0]
    n_nodes_per_elem = connectivite.shape[1]
    numEq = np.zeros((n_elem, 2*n_nodes_per_elem), dtype=int)
    for e in range(n_elem):
        for i in range(n_nodes_per_elem):
            numEq[e, 2*i]   = 2*connectivite[e, i];
            numEq[e, 2*i+1] = 2*connectivite[e, i]+1;
    return numEq

def assemblerMatriceRigidite(connectivite, coordonnees):

    n_elem  = connectivite.shape[0]
    n_nodes = coordonnees.shape[0]
    numEq = calculerNumerosEquations(connectivite)

    K = np.zeros((n_nodes*2, n_nodes*2))
    for e in range(n_elem):
        # On récupère les degrés de liberté de l'élément e
        ddl = numEq[e, :]
        # On récupère les noeuds de l'élément e
        connectivite_element = connectivite[e, :]
        # On calcule la matrice de rigidite locale de l'élément e
        K_locale = calculerMatriceRigiditeLocale(connectivite_element, coordonnees)
        # On assemble
        for i, gi in enumerate(ddl):
            for j, gj in enumerate(ddl):
                K[gi, gj] += K_locale[i, j]
    return K

#### Question

Construisez la matrice de rigidité et observez son profil (utilisez la fonction `plt.spy`)

In [ ]:
# Place your answer here

## Resolution du problème

#### Question 

Construire le vecteur des forces consistantes

In [ ]:
def calculerF(nodes):
    n_nodes = nodes.shape[0]
    # On connait à l'avance le nombre de ddl
    F = np.zeros((n_nodes, 2), dtype=float)
    # faire une boucle sur les noeuds et mettre la force au bon endroit
    # ....
        
    return F.ravel()

In [ ]:
# Place your answer here

#### Question

Construire le vecteur des blocages (fonction `calculerBlocages`)

In [ ]:
def calculerBlocages(nodes):
    n_nodes = nodes.shape[0]
    # On connait à l'avance le nombre de ddl bloqués
    blocages = np.zeros((n_nodes, 2), dtype=bool)
    # faire une boucle sur les noeuds et mettre True quand bloqué
    # ....
        
    return blocages.ravel()

blocages = calculerBlocages(points)
libres = np.logical_not(blocages)
libres

In [ ]:
# Place your answer here

#### Question 

Résoudre le problème et affichez la déformée (avec la fonction `plotMesh`)

In [ ]:
# Résolution du système Ku=F avec conditions limites
n_nodes = points.shape[0]
u = np.zeros(n_nodes*2)
u[libres] = np.linalg.solve(K[libres, :][:, libres], F[libres])

plotMesh(points+u.reshape(n_nodes, 2)*1000, conn, u)

# T3 Matrice de masse

5. En modifiant les routines ci-dessous, implémentez l'assemblage de la matrice de masse

Pour rappel, on a l'expression intégrale suivante:

$$[M] =  \iiint \rho (s,t) det(J(s, t)) [N(s,t)]^T [N(s,t)] ds dt$$

#### Question

Quel est l'ordre d'intégration nécéssaire pour réaliser cette intégrale ?

---

 **Place your answer here** 

 ---



#### Question

En vous inspirant de l'assemblage de la matrice de rigidité, faire l'assemblage de la matrice de masse.
   
*Indice: Quadrature de Gauss à trois points:*

| Quad points | \#1 | \#2 | \#3 |    
|:---| :---: | :---: | :---: |    
| Position $(s,t)$ | (1/6, 1/6) | (2/3, 1/6) | (1/6, 2/3) |
| Poids                       | 1/6 | 1/6 | 1/6|


In [ ]:

def calculerMatriceMasseLocale(connectivite_element, coordonnees):
    M_local = np.zeros((6, 6)) # 6x6 for T3 only

    # .....
    
    return M_local

def assemblerMatriceMasse(connectivite, coordonnees):

    n_elem  = connectivite.shape[0]
    n_nodes = coordonnees.shape[0]
    numEq = calculerNumerosEquations(connectivite)

    K = np.zeros((n_nodes*2, n_nodes*2))
    for e in range(n_elem):
        # On récupère les degrés de liberté de l'élément e
        ddl = numEq[e, :]
        # On récupère les noeuds de l'élément e
        connectivite_element = connectivite[e, :]
        # On calcule la matrice de rigidite locale de l'élément e
        M_locale = calculerMatriceMasseLocale(connectivite_element, coordonnees)
        # On assemble
        for i, gi in enumerate(ddl):
            for j, gj in enumerate(ddl):
                M[gi, gj] += M_locale[i, j]
    return M

In [ ]:
# Place your answer here

# Etude des modes propres

Pour étudier les modes propres, il faut résoudre une équation du type:

$$[M]^{-1}[K] \{v\} = \lambda \{v\} \qquad \Rightarrow \qquad [K] \{v\} = \omega^2 [M]\{v\}$$

avec la fréquence telle que $\omega = 2\pi f$

En python la routine [scipy.sparse.linalg.eigs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigs.html) peut résoudre ce type de problèmes sur des matrices creuses (plus rapide donc)

#### Question

Calculer les 5 premiers modes sans considérer les blocages. Affichez les avec la route `plotMesh`

In [ ]:
# Place your answer here

#### Question

Calculer les 5 premiers modes en considérerant les blocages. Affichez les avec la route `plotMesh`

In [ ]:
# Place your answer here